# Les espaces de stockage
Il y a plusieurs types de stockage sur les grappes nationales :
* Personnel (`/home`)
* Temporaire local (`$SLURM_TMPDIR`)
* Temporaire réseau (`/scratch`)
* Projet partagé (`/project`)
* *Nearline* - stockage de longue durée (`/nearline`)

Or, vos données ont plusieurs aspects :
* **Taille** : petits, gros, très gros fichiers
* **Nombre** : peu nombreux ou en très grand nombre
* **Transférabilité** : données regroupées et/ou compressées
* **Vie** : pendant une tâche, entre des tâches, d'un projet à l'autre, à archiver
* **Niveau d'accès** : données confidentielles, partagées, publiées

Le but de ce chapitre est de considérer les principaux
aspects nécessaires à la gestion des données actives
sur les différents espaces de stockage disponibles.

## Les différents types de stockage
Stockage accessible selon le type de noeud :
noeud de **connexion** (ou interactif) ou
noeud de **calcul** (CPU ou GPU) :

| Stockage accessible | Noeud de connexion | Noeud de calcul |
|---------------------|:------------------:|:---------------:|
|             `/home` |         Oui        |       Oui       |
|     `$SLURM_TMPDIR` |       **Non**      |       Oui       |
|          `/scratch` |         Oui        |       Oui       |
|          `/project` |         Oui        |       Oui       |
|         `/nearline` |         Oui        |     **Non**     |

À propos des [différents espaces de stockage](https://docs.alliancecan.ca/wiki/Storage_and_file_management/fr#Types_de_stockage).

### `$HOME` - Votre dossier personnel
```Bash
cd  # $HOME par défaut
ls -a
ls -la
ls -ld $HOME
```

* **Point d'entrée** par défaut lors d'une connexion à une grappe de calcul
* [Quota d'espace relativement petit](https://docs.alliancecan.ca/wiki/Storage_and_file_management/fr#Quotas_et_politiques), mais accepte un nombre de fichiers relativement grand
  * Idéal pour la [compilation et l'installation de logiciels](https://docs.alliancecan.ca/wiki/Installing_software_in_your_home_directory/fr)

### `$SLURM_TMPDIR` - Votre dossier temporaire local
```Bash
ls -ld $SLURM_TMPDIR
salloc  # À partir de login1

ls -ld $SLURM_TMPDIR
df -h $SLURM_TMPDIR
exit
```

* [Stockage local très rapide](https://docs.alliancecan.ca/wiki/Using_node-local_storage/fr), mais limité à la durée de la tâche de calcul
  * **Faible latence** à comparé *Lustre*
  * Grande bande-passante, en particulier pour les petits fichiers
  * **Données supprimées à la fin** de la tâche de calcul
  * Si plusieurs noeuds travaillent sur un même calcul parallèle,
    **chaque noeud a son propre dossier** `$SLURM_TMPDIR`
* Cas d'utilisation :
  * **Importation** de plusieurs **petits fichiers** qui seront
    utilisés à répétition lors d'un calcul
  * **Sauvegarde** de fichiers qui sont **constamment modifiés** -
    à exporter vers *Lustre* à la fin du calcul

### `$SCRATCH` - Votre espace réseau temporaire
```Bash
df -h /scratch
ls -ld $SCRATCH
```

* Espace de stockage réseau de [grande capacité](https://docs.alliancecan.ca/wiki/Storage_and_file_management/fr#Quotas_et_politiques) pour des **données temporaires**
  * **Pas** sauvegardé
  * [Purge mensuelle](https://docs.alliancecan.ca/wiki/Scratch_purging_policy/fr) pour les données âgées de plus de 60 jours
* Performance variable selon l'utilisation de l'ensemble des utilisateurs
* Cas d'utilisation :
  * Utilisation de données **pendant quelques jours** seulement
  * Stocker temporairement des **résultats en grand nombre** de fichiers
  * Stocker des résultats **intermédiaires** qui sont **trop gros**
    pour `/project`

### `/project` - Espace projet partagé
```Bash
ls -ld /project
ls -ld /project/def-sponsor00
ls -l /project/def-sponsor00
```

* Espace de stockage réseau de [petite à grande capacité](https://docs.alliancecan.ca/wiki/Storage_and_file_management/fr#Quotas_et_politiques) pour des **données de projet**
  * Un espace projet par défaut par groupe de recherche (sauf sur Niagara)
    * Petite augmentation d'espace projet **sur simple demande (jusqu'à 10 To)**
    * Quota plus élevé si [allocation spéciale](https://alliancecan.ca/fr/services/calcul-informatique-de-pointe/acces-aux-ressources/concours-pour-lallocation-de-ressources)
  * Sauvegardé **quotidiennement**
    * Nombre de fichiers relativement limité (500k par défaut)
* Données de projet :
  * Potentiellement **partagées** - [configuration des ACLs](https://docs.alliancecan.ca/wiki/Sharing_data/fr)
  * Durée de vie en fonction de la durée d'un projet
  * Typiquement plus importantes que les données temporaires
* Cas d'utilisation :
  * Stockage de jeux de données **réutilisés sur plusieurs mois et/ou par plusieurs personnes**
  * Stockage de **résultats finaux** coûteux à reproduire

### `/nearline` - Stockage très longue durée
Interface de stockage sur disque :
* On peut voir les fichiers avec la commande `ls`
* Les données les plus anciennes dans
  [`/nearline` sont probablement sur ruban](https://docs.alliancecan.ca/wiki/Using_nearline_storage/fr)
  * Voir les commandes pour [diagnostiquer l'état des fichiers](https://docs.alliancecan.ca/wiki/Using_nearline_storage/fr#Transf.C3.A9rer_des_donn.C3.A9es_.C3.A0_partir_de_.2Fnearline) dans `/nearline`

À considérer :
* D'une part, la migration de données sur ruban réduit l'espace
  utilisé sur les disques - économie d'argent pour le stockage
* D'autre part, chaque lecture de fichier migré sur ruban créera
  **une requête bloquante** causant un temps de réponse de quelques
  minutes à quelques heures (si le système d'archivage est surchargé
  de requêtes)
  * Voilà pourquoi il est impératif d'y sauvegarder un
    **faible nombre de très grands fichiers**
  * **À éviter** : y copier de nombreux petits fichiers avant de les
    regrouper dans un fichier d'archive

Cas d'utilisation :
  * Regrouper des fichiers provenant de `/project` ou de `/scratch`
  * Stockage de données importantes **qui ne seront pas utilisées
    pendant plusieurs mois**

## Caractéristiques de vos données
### Vie
Avec le temps, les données s'accumulent. Il devient alors nécessaire de surveiller l'espace utilisé, de même que le nombre de fichiers qui s'y trouvent.
```
du -s ~
find . | wc -l

df -h /project
df -hi /project
```

* **[La commande `diskusage_report`](https://docs.alliancecan.ca/wiki/Storage_and_file_management/fr#Introduction)**
  permet de générer un court rapport sur l'espace utilisé et le nombre
  de fichiers de chacun des espaces de stockage auxquels vous avez accès
* À chaque jour, un rapport de consommation de l'espace `/project` par utilisateur est produit :
  * Sur Béluga : dans `/project/.stats/<nom-allocation>`
  * Sur Cedar et Narval : dans `/project/.stats/<nom-allocation>.json`
  * **[La commande `diskusage_explorer`](https://docs.alliancecan.ca/wiki/Diskusage_Explorer/fr)**
    affiche un sommaire d'utilisation et permet de naviguer vers
    des sous-répertoires pour mieux analyser l'espace utilisé.
  * Information disponible sur demande pour Graham et Niagara

Avoir un bon plan de gestion des données actives permet de
faciliter le ménage (suppression ou archivage de fichiers).

#### Exemple de cycle de vie des données
![Pipeline 1](images/data-flow-1.png)

Description des différentes étapes :
* Téléchargement des données dans `/scratch`
  * Pour une utilisation de quelques jours à quelques semaines
  * Nul besoin de les protéger davantage
* Soumettre une série de tâches de calcul
  * Une tâche par fichier dans `dossier` dans la partition `/scratch`
* Le script de tâche est dans la partition `/home`
  * Utilisation des variables `$FIC` et `$SLURM_TMPDIR` pour copier le fichier à traiter localement sur le noeud de calcul
  * Se déplacer dans le dossier local
  * Y configurer un environnement Python
  * Exécuter le code Python présent dans `/home`, fournir le nom du fichier à traiter et rediriger toutes les sorties dans un fichier local
  * Rapatrier le fichier de résultats dans un dossier dans `/scratch`
* Post-traitement - traiter tous les fichiers de résultats afin de ne garder que l'essentiel dans `/project`

#### **Exercice** - Exécution d'un pipeline
* Étudiez les scripts :
  * `scripts/blastn-pipeline.sh`
  * `scripts/blastn-traitement.sh`
* Lancez le pipeline avec la commande :

```Bash
bash scripts/blastn-pipeline.sh
```

* Surveillez les tâches avec `squeue -u $USER`
* Trouver les fichiers créés dans :
  * `$SCRATCH/donnees` et
  * `$SCRATCH/donnees/res_prll`
* Lancez le post-traitement avec la commande :

```Bash
bash scripts/blastn-traitement.sh
```
* Trouvez le fichier TSV dans l'espace projet

### En cas de données inaccessibles
* Une professeure ou un professeur peut demander à ce que les données soient supprimées
* Par contre, pour obtenir l'accès aux données, il faut avoir le consentement de la personne qui les a isolées (probablement involontairement)
  * En cas d'absence de réponse, c'est la politique de l'université du groupe de recherche qui permettra ou non d'obtenir l'accès aux données
  
Dans tous les cas, il vaut mieux planifier, et ce,
dès l'importation de données sur les grappes de calcul.

## Points à retenir
* Le système de fichiers *Lustre* est optimisé pour des gros fichiers
  (+10 Mo)
  * Ne pas mettre trop de fichiers et dossiers dans un même répertoire
  (maximum 1000 items)
* Pour le transfert de données et l'utilisation du *Nearline*,
  il vaut mieux regrouper les données dans un fichier archive
  (comme Zip, DAR, etc.)
* Dans l'espace projet, il faut planifier qui devrait avoir accès à quoi et quand
* Pour optimiser les tâches, utiliser `$SLURM_TMPDIR`
* La commande `diskusage_report` pour un aperçu de l'espace utilisé
* Pour les données et codes critiques :
  * avoir une copie ailleurs et
  * utiliser un gestionnaire de versions.